# from tutorial

In [1]:
import xarray as xr
from pathlib import Path

In [2]:
data_2d_z2 =  '/g/data/qx55/germany_node/d3hp003.zarr/P1D_point_z10_atm.zarr'
ds_2 = xr.open_zarr(data_2d_z2)


In [3]:
pr = ds_2['pr']  # (time, cell)
pr_day = pr.isel(time=2) * 86400  # Convert from kg m-2 s-1 to mm/day

In [4]:
ds_2.pr

<xarray.DataArray 'pr' (time: 425, cell: 12582912)> Size: 21GB
dask.array<open_dataset-pr, shape=(425, 12582912), dtype=float32, chunksize=(4, 262144), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 3kB 2020-01-02 2020-01-03 ... 2021-03-01
Dimensions without coordinates: cell
Attributes:
    grid_mapping:        crs
    hiopy::time_method:  point
    hiopy::nnn:          4
    hiopy::enable:       True
    standard_name:       pr
    units:               kg m-2 s-1
    short_name:          
    long_name:           precipitation flux

In [5]:
ds_2.time

<xarray.DataArray 'time' (time: 425)> Size: 3kB
array(['2020-01-02T00:00:00.000000000', '2020-01-03T00:00:00.000000000',
       '2020-01-04T00:00:00.000000000', ..., '2021-02-27T00:00:00.000000000',
       '2021-02-28T00:00:00.000000000', '2021-03-01T00:00:00.000000000'],
      shape=(425,), dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 3kB 2020-01-02 2020-01-03 ... 2021-03-01
Attributes:
    axis:     T

# at different duration

In [6]:
data_2d_z2 = '/g/data/qx55/germany_node/d3hp003.zarr/PT1H_point_z10_atm.zarr'
ds_2 = xr.open_zarr(data_2d_z2)

# data_2d_z3 = '/g/data/qx55/germany_node/d3hp003.zarr/PT3H_point_z10_atm.zarr'
# ds_3 = xr.open_zarr(data_2d_z3)

data_2d_z4 = '/g/data/qx55/germany_node/d3hp003.zarr/P1D_mean_z10_atm.zarr'
ds_4 = xr.open_zarr(data_2d_z4)

# data_2d_z5 = '/g/data/qx55/germany_node/d3hp003.zarr/P1D_mean_z11_atm.zarr'
# ds_5 = xr.open_zarr(data_2d_z5)

In [7]:
ds_dict = {2:ds_2.pr, 4:ds_4.pr}

In [9]:

import matplotlib.pyplot as plt
import easygems.healpix as egh
import cartopy.crs as ccrs

In [ ]:
# Define projection
projection = ccrs.Robinson(central_longitude=0)

# Create subplots with projection
fig, axes = plt.subplots(2, 1, figsize=(12, 6),
                         subplot_kw={'projection': projection},
                         layout='constrained')

# Loop over dictionary items and plot each
for ax, (name, da) in zip(axes.flat, ds_dict.items()):
    # Sum over time and convert from kg/m²/s to mm (multiply by 86400 seconds per day)
    data = da.sum(dim='time') * 86400  

    ax.set_global()
    im = egh.healpix_show(data.values, ax=ax, vmin=0, vmax=1000, cmap='jet')  # adjust vmax if needed
    ax.coastlines()
    ax.set_title(f'UM N2560 RAL3 total precip = {name}')
    
    # Set zoomed extent (optional)
    ax.set_extent([110, 160, -45, -5], crs=ccrs.PlateCarree())

    # Add colorbar for each subplot
    cbar = fig.colorbar(im, ax=ax, orientation='vertical', shrink=0.7, pad=0.05)
    cbar.set_label("Total Precipitation (mm)")

plt.show()
